## Gathering Data

In [2]:
import pandas as pd
import tweepy as tw
import requests
import json
import os

In [3]:
# Import file with twitter posts as a pandas dataframe
feed = pd.read_csv('C:/Users/eyego/Downloads/twitter-archive-enhanced.csv', sep =',')

In [4]:
# Programatically download the datafile with breed predictions based on the tweet's image from a URL to insure reproducibility
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response = requests.get(url)
with open(os.path.join(url.split('/')[-1]), mode='wb') as file:
        file.write(response.content)
        
predictions = pd.read_csv('image-predictions.tsv', sep='\t')

In [5]:
# Twitter keys (delete before sharing)
consumer_key=""
consumer_secret=""
access_token=""
access_token_secret=""

In [6]:
# query the Twitter API for each tweet's JSON data using Python's Tweepy
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

In [7]:
# Dataframe with the tweet ids in twitter_archive-enhanced.csv
tweet_ids = feed.tweet_id

In [8]:
# retrieve the full JSON for the status of each id in twitter-archive-enhanced.csv
# This script takes over 35 minutes to run
failed_id = []
with open('tweet_json.txt', 'w', encoding='UTF-8') as json_file:
    for id_ in tweet_ids:
        try:
            tweet = api.get_status(id_)
            tweet = tweet._json
            json.dump(tweet, json_file)
            json_file.write('\n')
        except:
            print('Failed retrieving tweet with id ',id_)
            failed_id.append(id_)

Failed retrieving tweet with id  892420643555336193
Failed retrieving tweet with id  892177421306343426
Failed retrieving tweet with id  891815181378084864
Failed retrieving tweet with id  891689557279858688
Failed retrieving tweet with id  891327558926688256
Failed retrieving tweet with id  891087950875897856
Failed retrieving tweet with id  890971913173991426
Failed retrieving tweet with id  890729181411237888
Failed retrieving tweet with id  890609185150312448
Failed retrieving tweet with id  890240255349198849
Failed retrieving tweet with id  890006608113172480
Failed retrieving tweet with id  889880896479866881
Failed retrieving tweet with id  889665388333682689
Failed retrieving tweet with id  889638837579907072
Failed retrieving tweet with id  889531135344209921
Failed retrieving tweet with id  889278841981685760
Failed retrieving tweet with id  888917238123831296
Failed retrieving tweet with id  888804989199671297
Failed retrieving tweet with id  888554962724278272
Failed retri

In [9]:
# Read the contents of the created file and create a dataframe with the fields of interest 

df = []
with open('tweet_json.txt') as f:
    for line in f:
        tweet = (json.loads(line))
        tweet_id = tweet['id']
        retweet_count = tweet['retweet_count']
        favorite_count = tweet['favorite_count']
        create_date = tweet['created_at']
        df.append({'retweet_count' : retweet_count,
                  'favorite_count' : favorite_count,
                  'create_date' : create_date,
                  'tweet_id' : tweet_id})
        
more_data = pd.DataFrame(df, columns = ['tweet_id', 'retweet_count', 'favorite_count', 'create_date'])

## Assessing Data
lets start with visual assesment

In [10]:
# assessing the file with the retrieved data for WeRateDogs tweets
feed.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


In [11]:
# assessing the file with the breed predictions for each dog
predictions.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


Now lets do Programatical Assessment
After assessing visually the three datasets and identifying several tidiness issues, we now go deeper using Programatic Assessment:

In [12]:
feed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2356 non-null   int64  
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2356 non-null   object 
 4   source                      2356 non-null   object 
 5   text                        2356 non-null   object 
 6   retweeted_status_id         181 non-null    float64
 7   retweeted_status_user_id    181 non-null    float64
 8   retweeted_status_timestamp  181 non-null    object 
 9   expanded_urls               2297 non-null   object 
 10  rating_numerator            2356 non-null   int64  
 11  rating_denominator          2356 non-null   int64  
 12  name                        2356 non-null   object 
 13  doggo                       2356 

The missing data for retweets and replies is fine, since we are not interested in those. 59 expanded URLs are missing.

In [13]:
predictions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   tweet_id  2075 non-null   int64  
 1   jpg_url   2075 non-null   object 
 2   img_num   2075 non-null   int64  
 3   p1        2075 non-null   object 
 4   p1_conf   2075 non-null   float64
 5   p1_dog    2075 non-null   bool   
 6   p2        2075 non-null   object 
 7   p2_conf   2075 non-null   float64
 8   p2_dog    2075 non-null   bool   
 9   p3        2075 non-null   object 
 10  p3_conf   2075 non-null   float64
 11  p3_dog    2075 non-null   bool   
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [14]:
more_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   tweet_id        0 non-null      object
 1   retweet_count   0 non-null      object
 2   favorite_count  0 non-null      object
 3   create_date     0 non-null      object
dtypes: object(4)
memory usage: 0.0+ bytes


In [15]:
# We test whether the missing expanded URLs are a problem.
# If there is a link in the tweet's text and no expanded URL then there is an issue

explore = feed[feed['expanded_urls'].isnull()]
explore['text'].str.contains('http').sum()

0

In [16]:
# value counts for the source column
feed.source.value_counts()

<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>     2221
<a href="http://vine.co" rel="nofollow">Vine - Make a Scene</a>                          91
<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>                       33
<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>      11
Name: source, dtype: int64

In [17]:
# every numerator not other than 10-14 in the dogs rating is a possible quality issue
feed.rating_numerator.value_counts().sort_index(ascending=False)

1776      1
960       1
666       1
420       2
204       1
182       1
165       1
144       1
143       1
121       1
99        1
88        1
84        1
80        1
75        2
60        1
50        1
45        1
44        1
27        1
26        1
24        1
20        1
17        1
15        2
14       54
13      351
12      558
11      464
10      461
9       158
8       102
7        55
6        32
5        37
4        17
3        19
2         9
1         9
0         2
Name: rating_numerator, dtype: int64

In [18]:
# every denominator not equal to 10 in the dogs rating is a possible quality issue
feed.rating_denominator.value_counts().sort_index(ascending=False)

170       1
150       1
130       1
120       1
110       1
90        1
80        2
70        1
50        3
40        1
20        2
16        1
15        1
11        3
10     2333
7         1
2         1
0         1
Name: rating_denominator, dtype: int64

In [19]:
# we check for duplicated IDs in the feeds dataframe
feed[feed.tweet_id.duplicated()]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo


In [20]:
predictions.describe()

,tweet_id,img_num,p1_conf,p2_conf,p3_conf
count,2.075000e+03,2075.000000,2075.000000,2.075000e+03,2.075000e+03
mean,7.384514e+17,1.203855,0.594548,1.345886e-01,6.032417e-02
std,6.785203e+16,0.561875,0.271174,1.006657e-01,5.090593e-02
min,6.660209e+17,1.000000,0.044333,1.011300e-08,1.740170e-10
25%,6.764835e+17,1.000000,0.364412,5.388625e-02,1.622240e-02
50%,7.119988e+17,1.000000,0.588230,1.181810e-01,4.944380e-02
75%,7.932034e+17,1.000000,0.843855,1.955655e-01,9.180755e-02
max,8.924206e+17,4.000000,1.000000,4.880140e-01,2.734190e-01


The predictions dataframe is the only one with numerical data. With .describe() we can check whether the ranges, means and quartiles make sense. In this test, we correctly see that the probability for the first guess has the largest value. We also see that the highest probability is equal to 1, which is expected.

In [21]:
# Checking for duplicated columns by creating a list of every column in every dataset
all_columns = pd.Series(list(feed) + list(predictions) + list(more_data))
# all_columns
all_columns[all_columns.duplicated()]

17    tweet_id
29    tweet_id
dtype: object

The only duplicated column is tweet_id, which is ok. This is how tidiness is assessed programatically.

### Issues

Quality

twitter-archive-enhanced.csv

- The extracted rating numerators are wrong when they are decimals like 13.5. Status:solved.
- source column has innecesary HTML code. Status: solved
- some tweets are retweets or replies. Status: solved
- timestamp column is not in datatime format. Status: solved
- The rating for dog in tweet with id 716439118184652801 is wrong. Status: solved
- The rating for dog in tweet with id 722974582966214656 is wrong. Status: solved
- The rating for dogs in tweet with id 682962037429899265 is wrong. Status: solved
- Many dog names are wrong, including hose with id 740373189193256964 and 770414278348247044. Many names are just other words in English. Status: solved
- The dog stage for dog in tweet with id 854010172552949760 is wrong. Status: solved
- some rows have several identical values in the expanded_url column concatenated by a comma. Status: solved
- Tweet_id fields in the three datasets are stored as numeric values and should be strings. Status: solved
- The values for the column "floffer" should not be capitalized to maintain integrity with the others. Status: solved

more_data dataframe

- Retweet and favorite count columns have float as datatype and should be int. Status: solved

Tidiness

twitter-archive-enhanced.csv

- Dog "stage" is spread in four columns. Status: solved
- Data about dogs and tweets in a same dataset. Status: solved
- The dog's raiting appears both in the "text" column and the rating numerator/denominator columns. Status: will not be solved here
- The link to the dog's photo is part of the status text and should have own column. Status: will not be solved here

image-predictions.tsv

- Predictions are spread in three columns. Status: solved
- Confidence intervals are spread in three columns. Status: solved
- Dog tests are spread in three columns. Status: solved

additional twitter data

- Create date exists already in twitter-archive-enhanced.tsv. Status: solved
- The dataset is split from the main dataset describing tweets. Status: solved


## Cleaning Data

In [22]:
# create copies of the datasets
feed_clean = feed.copy()
predictions_clean = predictions.copy()
more_data_clean = more_data.copy()

In [23]:
feed_clean = feed_clean[(feed_clean['in_reply_to_status_id'].isna() == True)  & (feed_clean['retweeted_status_id'].isna() == True)]

feed_clean = feed_clean.drop(['in_reply_to_status_id',
                              'in_reply_to_user_id',
                              'retweeted_status_id',
                              'retweeted_status_user_id',
                              'retweeted_status_timestamp'],
                            axis = 1)

In [24]:
#test
feed_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2097 entries, 0 to 2355
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   tweet_id            2097 non-null   int64 
 1   timestamp           2097 non-null   object
 2   source              2097 non-null   object
 3   text                2097 non-null   object
 4   expanded_urls       2094 non-null   object
 5   rating_numerator    2097 non-null   int64 
 6   rating_denominator  2097 non-null   int64 
 7   name                2097 non-null   object
 8   doggo               2097 non-null   object
 9   floofer             2097 non-null   object
 10  pupper              2097 non-null   object
 11  puppo               2097 non-null   object
dtypes: int64(3), object(9)
memory usage: 213.0+ KB


In [25]:
#Change datatype and use regex to extract the ratings numerators again.
feed_clean['rating_numerator'] = feed_clean['rating_numerator'].astype('float')
feed_clean['rating_numerator'] = feed_clean['text'].str.extract('(\d+\.*\d*\/\d+)', expand=False).str.split('/').str[0]

In [26]:
#Test
print(feed_clean.rating_numerator.dtype)

object


In [27]:
feed_clean.loc[feed_clean['tweet_id'] == 883482846933004288, 'rating_numerator']

45    13.5
Name: rating_numerator, dtype: object

Data about dogs and tweets in a same dataset

 In order to have observational units in their own table, we need to split the feed_clean data-set into a tweets table and a dogs table. This is a design decision because there are many several tweets that include information and ratings for more than one dog. Ratings and dog stages refer to dogs, whereas retweets an likes are related to the tweets.

In [28]:
dogs = feed_clean[['tweet_id', 'name', 'doggo', 'floofer', 'pupper', 'puppo', 'rating_numerator', 'rating_denominator']].copy()
tweets = feed_clean.drop(['name', 'doggo', 'floofer', 'pupper', 'puppo', 'rating_numerator', 'rating_denominator'], axis=1)

In [29]:
#Test
print(tweets.columns)
print(dogs.columns)

Index(['tweet_id', 'timestamp', 'source', 'text', 'expanded_urls'], dtype='object')
Index(['tweet_id', 'name', 'doggo', 'floofer', 'pupper', 'puppo',
       'rating_numerator', 'rating_denominator'],
      dtype='object')


Dog "stage" is spread in four columns

We have values as variables in the now called dogs table. We will melt the columns doggo, floofer, pupper, puppo into a new variable called dog_stage. Before melting, create a new column to identify those dogs with dog stage 'unknown'

In [30]:
# we want to create first an extra column to tag those dogs without a recognized stage. 
# This will help cleaning the dataset afterwards
def u(row):
    if row ['doggo'] == 'None' and row ['floofer'] == 'None' and row ['pupper'] == 'None' and row ['puppo'] == 'None':
        val = 'unknown'
    else:
        val ='None'
    return val

dogs['unknown'] = dogs.apply(u, axis=1)

In [31]:
dogs.sample()

,tweet_id,name,doggo,floofer,pupper,puppo,rating_numerator,rating_denominator,unknown
57,881268444196462592,Elliot,None,None,None,None,12,10,unknown


In [32]:
# we melt the different dog stages into a new column calle 'dog_stage'
dogs = pd.melt(dogs, id_vars =['tweet_id', 'name', 'rating_numerator','rating_denominator'],
                     value_vars = ['doggo', 'floofer', 'pupper', 'puppo', 'unknown'],
                     var_name = 'dog_stage', 
                    value_name = 'value')

In [33]:
# clean the duplicated rows created in the preious process and drop the 'value' variable
dogs = dogs[dogs['value']!= 'None']
dogs = dogs.drop('value', axis=1)

In [34]:
#Test
dogs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2108 entries, 9 to 10484
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   tweet_id            2108 non-null   int64 
 1   name                2108 non-null   object
 2   rating_numerator    2108 non-null   object
 3   rating_denominator  2108 non-null   int64 
 4   dog_stage           2108 non-null   object
dtypes: int64(2), object(3)
memory usage: 98.8+ KB


It is normal that now we have more records (2,108 vs 2,097), because some tweets referred to more than one dog and included more than one dog stage

The different predictions, the confidence and the test whether the prediction is a dog or not need to be melted into four columns: prediction_number, prediction, confidence and dog

In [35]:
# This script iterates the melting process for the three categories of data: prediction, confidence and dog test. 
# at the end we have a clean tidy dataset.

# create 'prediction_number' and 'prediction' columns
predictions_clean = pd.melt(predictions_clean, id_vars = ['tweet_id', 'jpg_url', 'img_num', 'p1_conf', 'p1_dog',
       'p2_conf', 'p2_dog', 'p3_conf', 'p3_dog'],
               var_name = 'prediction_number',
               value_name = 'prediction')

# create 'confidence' column
predictions_clean = pd.melt(predictions_clean, id_vars = ['tweet_id', 'jpg_url', 'img_num', 'p1_dog',
       'p2_dog', 'p3_dog', 'prediction_number', 'prediction'],
               var_name = 'to_delete',
               value_name = 'confidence')

# remove newly created duplicated rows
predictions_clean = predictions_clean[predictions_clean['prediction_number'] == predictions_clean['to_delete'].str[:2]]
# remove unnecesary column
predictions_clean = predictions_clean.drop('to_delete', axis=1)

# create 'dog' column
predictions_clean = pd.melt(predictions_clean, id_vars = ['tweet_id', 'jpg_url', 'img_num','prediction_number', 'prediction', 'confidence'],
               var_name = 'to_delete2',
               value_name = 'dog')

# remove newly created duplicated rows
predictions_clean = predictions_clean[predictions_clean['prediction_number'] == predictions_clean['to_delete2'].str[:2]]

# remove unnecesary column
predictions_clean = predictions_clean.drop('to_delete2', axis=1)
# remove 'p' from prediction number
predictions_clean['prediction_number'] = predictions_clean['prediction_number'].str[1]

In [36]:
#Test
predictions_clean.sort_values(by = ['tweet_id', 'prediction_number']).head(9)

,tweet_id,jpg_url,img_num,prediction_number,prediction,confidence,dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,1,Welsh_springer_spaniel,0.465074,True
8300,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,2,collie,0.156665,True
16600,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,3,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,1,redbone,0.506826,True
8301,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,2,miniature_pinscher,0.074192,True
16601,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,3,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,1,German_shepherd,0.596461,True
8302,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,2,malinois,0.138584,True
16602,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,3,bloodhound,0.116197,True


Create date not needed in more_data_clean (field already exists in tweets dataframe)

In [37]:
more_data_clean = more_data_clean.drop('create_date', axis = 1)

In [38]:
more_data_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   tweet_id        0 non-null      object
 1   retweet_count   0 non-null      object
 2   favorite_count  0 non-null      object
dtypes: object(3)
memory usage: 0.0+ bytes


In [39]:
tweets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2097 entries, 0 to 2355
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   tweet_id       2097 non-null   int64 
 1   timestamp      2097 non-null   object
 2   source         2097 non-null   object
 3   text           2097 non-null   object
 4   expanded_urls  2094 non-null   object
dtypes: int64(1), object(4)
memory usage: 98.3+ KB


In [40]:
# Iterate through each row and extract the source's text with beautiful soup

from bs4 import BeautifulSoup as bs

new_source = []
for line, row in tweets.iterrows():
    soup = bs(row.source)
    x = soup.find('a').contents[0]
    new_source.append(x)
    
tweets['source'] = new_source

In [41]:
#Test
pd.Series(new_source).value_counts()

Twitter for iPhone     1964
Vine - Make a Scene      91
Twitter Web Client       31
TweetDeck                11
dtype: int64

In [42]:
tweets['timestamp'] = pd.to_datetime(tweets['timestamp'])

In [43]:
#Test
tweets.dtypes

tweet_id                       int64
timestamp        datetime64[ns, UTC]
source                        object
text                          object
expanded_urls                 object
dtype: object

- The rating for dog in tweet with id 716439118184652801 is wrong
- The rating for dog in tweet with id 722974582966214656 is wrong
- The rating for dogs in tweet with id 682962037429899265 is wrong

In [44]:
dogs.loc[dogs['tweet_id'] == 716439118184652801, 'rating_numerator'] = 11
dogs.loc[dogs['tweet_id'] == 716439118184652801, 'rating_denominator'] = 10

dogs.loc[dogs['tweet_id'] == 722974582966214656, 'rating_numerator'] = 13
dogs.loc[dogs['tweet_id'] == 722974582966214656, 'rating_denominator'] = 10

dogs.loc[dogs['tweet_id'] == 682962037429899265, 'rating_numerator'] = 10
dogs.loc[dogs['tweet_id'] == 682962037429899265, 'rating_denominator'] = 10

In [45]:
ids = [716439118184652801, 722974582966214656, 682962037429899265]
dogs.loc[dogs['tweet_id'].isin(ids), ['tweet_id','rating_numerator', 'rating_denominator']]

,tweet_id,rating_numerator,rating_denominator
9336,722974582966214656,13,10
9373,716439118184652801,11,10
9814,682962037429899265,10,10


Many dog names are wrong, including hose with id 740373189193256964 and 770414278348247044. Many names are just other words in English

In [48]:
#Look up and correct the dog's names in the dogs table
dogs.loc[dogs['tweet_id'] == 887517139158093824, 'name'] = 'None'
dogs.loc[dogs['tweet_id'] == 770414278348247044, 'name'] = 'Al Cabone'

In [49]:
ids = [887517139158093824, 770414278348247044]
dogs.loc[dogs['tweet_id'].isin(ids), ['tweet_id','name']]

,tweet_id,name
8409,887517139158093824,None
9020,770414278348247044,Al Cabone


In [50]:
dogs.loc[dogs['tweet_id'] == 854010172552949760, 'dog_stage'] = 'floofer'
dogs.drop(2258, inplace = True)

In [51]:
#test

dogs.loc[dogs['tweet_id'] == 854010172552949760 , ['tweet_id','dog_stage']]

,tweet_id,dog_stage
161,854010172552949760,floofer


In [53]:
#Split value to keep only the first URL in the column expanded_urls in the tweets dataset. Rename column to expanded_url
tweets['expanded_urls'] = tweets['expanded_urls'].str.split(',', expand=True)[0]

tweets = tweets.rename(index=str, columns={"expanded_urls": "expanded_url"})

In [54]:
#test
tweets.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2097 entries, 0 to 2355
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype              
---  ------        --------------  -----              
 0   tweet_id      2097 non-null   int64              
 1   timestamp     2097 non-null   datetime64[ns, UTC]
 2   source        2097 non-null   object             
 3   text          2097 non-null   object             
 4   expanded_url  2094 non-null   object             
dtypes: datetime64[ns, UTC](1), int64(1), object(3)
memory usage: 98.3+ KB


In [55]:
tweets['expanded_url'].str.contains(',').sum()

0

Change the numeric datatypes for the tweet_id variable from int to object in the three datasets

In [58]:
tweets['tweet_id'] = tweets['tweet_id'].astype(str)
dogs['tweet_id'] = dogs['tweet_id'].astype(str)
predictions['tweet_id'] = predictions['tweet_id'].astype(str)
print(tweets.tweet_id.dtype)
print(dogs.tweet_id.dtype)
print(predictions.tweet_id.dtype)

object
object
object


Change the datatypes for the retweet_count and favorite_count variables from float to int in tweets dataset

In [60]:
tweets.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2097 entries, 0 to 2355
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype              
---  ------        --------------  -----              
 0   tweet_id      2097 non-null   object             
 1   timestamp     2097 non-null   datetime64[ns, UTC]
 2   source        2097 non-null   object             
 3   text          2097 non-null   object             
 4   expanded_url  2094 non-null   object             
dtypes: datetime64[ns, UTC](1), object(4)
memory usage: 98.3+ KB


This concludes my cleaning of the identified issues for the project's datasets.

In [61]:
# save the three tables with independent observational units
tweets.to_csv('tweets.csv', sep=';', index=False)
dogs.to_csv('dogs.csv', sep=';', index=False)
predictions.to_csv('predictions.csv', sep=';', index=False)

In [62]:
# create and save a master dataset. The key needs to be set to 'tweet_id' in order for the join to work.
twitter_archive_master = dogs.set_index('tweet_id').join(tweets.set_index('tweet_id'), on='tweet_id', how='left')
twitter_archive_master.to_csv('twitter_archive_master.csv', sep=';', index=False)

### Analysis

With a cleaned dataset we can now proceed to perform some basic analysis to obtain insights from the data. I will explore the data to try and solve the following questions:

- Are retweets and replies correlated?
- Which are the most common dog stage?
- What dog stage receives more likes and retweets?
- What dog stage receives the best ratings?
- What's the distribution for the retweet count?

In [64]:
import altair as alt

In [65]:
tweets_master = pd.read_csv('twitter_archive_master.csv', sep=";")

In [66]:
chart = alt.Chart(tweets).mark_point()

In [70]:
alt.Chart(tweets_master[tweets_master['dog_stage']!= 'unknown']).mark_bar(color='green', opacity=0.5).encode(
    alt.Y('dog_stage', axis=alt.Axis(title='Dog Stage')),
    x='count()'
).properties(title = 'Most dogs rated are puppers')

alt.Chart(...)

In [74]:
alt.Chart(tweets_master[tweets_master['dog_stage']!= 'unknown']).mark_bar(color='firebrick', opacity=0.5).encode(
    alt.Y('dog_stage', axis=alt.Axis(title='Dog Stage')),
    alt.X('average(rating_numerator)', axis=alt.Axis(title='Average Rating'))
).properties(title = 'Best rated dogs are puppos')

alt.Chart(...)

### Resources
- Pandas sort_values(): http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.sort_values.html
- Sort values issue in SO: https://stackoverflow.com/questions/44123874/dataframe-object-has-no-attribute-sort
- Reading and writing JSON to a file: https://stackabuse.com/reading-and-writing-json-to-a-file-in-python/
- SQL Alchemy: https://www.sqlalchemy.org/
- Pillow Documentation: https://pillow.readthedocs.io/en/stable/
- API tutorial of Mediawiki: https://www.mediawiki.org/wiki/API:Tutorial
- Requests Documentation: http://docs.python-requests.org/en/master/user/intro/
- Glob documentation: https://docs.python.org/3/library/glob.html
- Python Files I/O: https://www.tutorialspoint.com/python/python_files_io.htm
- Assessing a tweet with only its ID: https://www.bram.us/2017/11/22/accessing-a-tweet-using-only-its-id-and-without-the-twitter-api/
- Reading and writing files in Python: https://www.pythonforbeginners.com/files/reading-and-writing-files-in-python
- Test for a pattern in a Pandas dataframe: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.str.contains.html
- Tidy data: https://cran.r-project.org/web/packages/tidyr/vignettes/tidy-data.html
- Imputation concepts for filling missing data: https://en.wikipedia.org/wiki/Imputation_(statistics)
- Melt function in pandas: https://pandas.pydata.org/pandas-docs/version/0.23.4/generated/pandas.DataFrame.melt.html#pandas.DataFrame.melt
- Commenting several lines of code in Jupyter Notebooks: https://stackoverflow.com/questions/29885371/how-do-i-comment-out-multiple-lines-in-jupyter-ipython-notebook
- Extracting specific columns into a new dataframe: https://stackoverflow.com/questions/34682828/extracting-specific-selected-columns-to-new-dataframe-as-a-copy
- How to iterate through a pandas series: https://stackoverflow.com/questions/16476924/how-to-iterate-over-rows-in-a-dataframe-in-pandas and https://stackoverflow.com/questions/43222878/iterate-over-pandas-dataframe-and-update-the-value-attributeerror-cant-set-a
- Work with strings in pandas: https://pandas.pydata.org/pandas-docs/stable/text.htm
- Merging datasets in pandas: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html
- Pandas .isin() function: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.isin.html
- Using .loc and .iloc to slice in pandas: https://www.shanelynn.ie/select-pandas-dataframe-rows-and-columns-using-iloc-loc-and-ix/